# Imports and Global Settings

In [1]:
import sys
import datetime
import pandas as pd
from sqlalchemy import create_engine

sys.path.append('../')
from ..passkeys import RDS_ENDPOINT, RDS_PASSWORD

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Data Inbound

In [2]:
username = 'postgres'
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = 'nba_betting'
port = '5432'

In [3]:
connection= create_engine(f'postgresql+psycopg2://{username}:{password}@{endpoint}/{database}').connect()

In [4]:
current_odds = pd.read_sql_table('dfc_covers_odds', connection)
current_game_results = pd.read_sql_table('dfc_covers_game_results', connection)
historic_odds = pd.read_sql_table('dfh_covers_odds', connection)

In [6]:
traditional = pd.read_sql_table('traditional', connection)
advanced = pd.read_sql_table('advanced', connection)
four_factors = pd.read_sql_table('four_factors', connection)
misc = pd.read_sql_table('misc', connection)
scoring = pd.read_sql_table('scoring', connection)
opponent = pd.read_sql_table('opponent', connection)
speed_distance = pd.read_sql_table('speed_distance', connection)
shooting = pd.read_sql_table('shooting', connection)
opponent_shooting = pd.read_sql_table('opponent_shooting', connection)
hustle = pd.read_sql_table('hustle', connection)

# Standardize Dates

In [7]:
def date_helper(old_date):
    if isinstance(old_date, datetime.datetime): 
        new_date = old_date.strftime('%Y%m%d')
    else:
        new_date = old_date.replace('-', '')
    return pd.to_datetime(new_date)

In [8]:
traditional['date'] = traditional['date'].apply(date_helper)
advanced['date'] = advanced['date'].apply(date_helper)
four_factors['date'] = four_factors['date'].apply(date_helper)
misc['date'] = misc['date'].apply(date_helper)
scoring['date'] = scoring['date'].apply(date_helper)
opponent['date'] = opponent['date'].apply(date_helper)
speed_distance['date'] = speed_distance['date'].apply(date_helper)
shooting['date'] = shooting['date'].apply(date_helper)
opponent_shooting['date'] = opponent_shooting['date'].apply(date_helper)
hustle['date'] = hustle['date'].apply(date_helper)

In [9]:
current_odds['date'] = pd.to_datetime(current_odds['date'])
current_game_results['date'] = pd.to_datetime(current_game_results['date'])
historic_odds['date'] = pd.to_datetime(historic_odds['date'])

# Standardize Team Names

In [10]:
    team_full_name_map = {
        "Washington Wizards": "WAS",
        "Brooklyn Nets": "BKN",
        "Chicago Bulls": "CHI",
        "Miami Heat": "MIA",
        "Cleveland Cavaliers": "CLE",
        "Philadelphia 76ers": "PHI",
        "New York Knicks": "NYK",
        "Charlotte Hornets": "CHA",
        "Boston Celtics": "BOS",
        "Toronto Raptors": "TOR",
        "Milwaukee Bucks": "MIL",
        "Atlanta Hawks": "ATL",
        "Indiana Pacers": "IND",
        "Detroit Pistons": "DET",
        "Orlando Magic": "ORL",
        "Golden State Warriors": "GSW",
        "Phoenix Suns": "PHX",
        "Dallas Mavericks": "DAL",
        "Denver Nuggets": "DEN",
        "Los Angeles Clippers": "LAC",
        "LA Clippers": "LAC",
        "Utah Jazz": "UTA",
        "Los Angeles Lakers": "LAL",
        "Memphis Grizzlies": "MEM",
        "Portland Trail Blazers": "POR",
        "Sacramento Kings": "SAC",
        "Oklahoma City Thunder": "OKC",
        "Minnesota Timberwolves": "MIN",
        "San Antonio Spurs": "SAS",
        "New Orleans Pelicans": "NOP",
        "Houston Rockets": "HOU",
        "Charlotte Bobcats": "CHA",
        "New Orleans Hornets": "NOP",
        "New Jersey Nets": "BKN",
        "Seattle SuperSonics": "OKC",
        "New Orleans/Oklahoma City Hornets": "NOP",
    }

    team_abrv_map = {
        "BK": "BKN",
        "BKN": "BKN",
        "BOS": "BOS",
        "MIL": "MIL",
        "ATL": "ATL",
        "CHA": "CHA",
        "CHO": "CHA",
        "CHI": "CHI",
        "CLE": "CLE",
        "DAL": "DAL",
        "DEN": "DEN",
        "DET": "DET",
        "GS": "GSW",
        "GSW": "GSW",
        "HOU": "HOU",
        "IND": "IND",
        "LAC": "LAC",
        "LAL": "LAL",
        "MEM": "MEM",
        "MIA": "MIA",
        "MIN": "MIN",
        "NO": "NOP",
        "NOP": "NOP",
        "NY": "NYK",
        "NYK": "NYK",
        "OKC": "OKC",
        "ORL": "ORL",
        "PHI": "PHI",
        "PHO": "PHX",
        "PHX": "PHX",
        "POR": "POR",
        "SA": "SAS",
        "SAS": "SAS",
        "SAC": "SAC",
        "TOR": "TOR",
        "UTA": "UTA",
        "WAS": "WAS",
    }

    team_short_name_map = {
        "Nets": "BKN",
        "Celtics": "BOS",
        "Bucks": "MIL",
        "Hawks": "ATL",
        "Hornets": "CHA",
        "Bulls": "CHI",
        "Cavaliers": "CLE",
        "Mavericks": "DAL",
        "Nuggets": "DEN",
        "Pistons": "DET",
        "Warriors": "GSW",
        "Rockets": "HOU",
        "Pacers": "IND",
        "Clippers": "LAC",
        "Lakers": "LAL",
        "Grizzlies": "MEM",
        "Heat": "MIA",
        "Timberwolves": "MIN",
        "Pelicans": "NOP",
        "Knicks": "NYK",
        "Thunder": "OKC",
        "Magic": "ORL",
        "76ers": "PHI",
        "Suns": "PHX",
        "Trail Blazers": "POR",
        "Spurs": "SAS",
        "Kings": "SAC",
        "Raptors": "TOR",
        "Jazz": "UTA",
        "Wizards": "WAS",
    }

    teams = dict(
        team_full_name_map.items()
        | team_abrv_map.items()
        | team_short_name_map.items()
    )

In [11]:
traditional['team'] = traditional['team'].map(teams)
advanced['team'] = advanced['team'].map(teams)
four_factors['team'] = four_factors['team'].map(teams)
misc['team'] = misc['team'].map(teams)
scoring['team'] = scoring['team'].map(teams)
opponent['team'] = opponent['team'].map(teams)
speed_distance['team'] = speed_distance['team'].map(teams)
shooting['team'] = shooting['team'].map(teams)
opponent_shooting['team'] = opponent_shooting['team'].map(teams)
hustle['team'] = hustle['team'].map(teams)

In [12]:
current_odds['team'] = current_odds['home_team_short_name'].map(teams)

In [13]:
current_odds['opponent'] = current_odds['away_team_short_name'].map(teams)

In [14]:
current_game_results['team'] = current_game_results['home_team'].map(teams)

In [15]:
current_game_results['opponent'] = current_game_results['away_team'].map(teams)

# Remove Duplicates and Empty Rows

In [16]:
traditional = traditional.dropna(thresh=3)
traditional = traditional.drop_duplicates(subset=['date', 'team'], keep='last')
advanced = advanced.dropna(thresh=3)
advanced = advanced.drop_duplicates(subset=['date', 'team'], keep='last')
four_factors = four_factors.dropna(thresh=3)
four_factors = four_factors.drop_duplicates(subset=['date', 'team'], keep='last')
misc = misc.dropna(thresh=3)
misc = misc.drop_duplicates(subset=['date', 'team'], keep='last')
scoring = scoring.dropna(thresh=3)
scoring = scoring.drop_duplicates(subset=['date', 'team'], keep='last')
opponent = opponent.dropna(thresh=3)
opponent = opponent.drop_duplicates(subset=['date', 'team'], keep='last')
speed_distance = speed_distance.dropna(thresh=3)
speed_distance = speed_distance.drop_duplicates(subset=['date', 'team'], keep='last')
shooting = shooting.dropna(thresh=3)
shooting = shooting.drop_duplicates(subset=['date', 'team'], keep='last')
opponent_shooting = opponent_shooting.dropna(thresh=3)
opponent_shooting = opponent_shooting.drop_duplicates(subset=['date', 'team'], keep='last')
hustle = hustle.dropna(thresh=3)
hustle = hustle.drop_duplicates(subset=['date', 'team'], keep='last')

In [17]:
current_game_results = current_game_results.drop_duplicates()

# Combine Data

In [18]:
historic_odds['pred_date'] = historic_odds['date'] - pd.DateOffset(1)

In [19]:
current_odds['pred_date'] = current_odds['date'] - pd.DateOffset(1)

In [20]:
historic_df = historic_odds.copy()

In [21]:
for stat_group in [traditional, advanced, four_factors, misc, scoring, opponent,
                   speed_distance, shooting, opponent_shooting, hustle]:
    historic_df = historic_df.merge(stat_group,
                                   how='left',
                                   left_on=['team', 'pred_date'],
                                   right_on=['team', 'date'],
                                   suffixes=(None, '_nba'),
                                   validate='1:m')
    historic_df = historic_df.drop(columns=['date_nba', 'date_opp', 'team_opp'], errors='ignore')
    historic_df = historic_df.merge(stat_group,
                                    how='left',
                                    left_on=['opponent', 'pred_date'],
                                    right_on=['team', 'date'],
                                    suffixes=(None, '_opp'),
                                    validate='1:m')
    historic_df = historic_df.drop(columns=['date_nba', 'date_opp', 'team_opp'], errors='ignore')

In [22]:
side_df = current_odds.merge(current_game_results,
                                   how='left',
                                   left_on=['team', 'date'],
                                   right_on=['team', 'date'],
                                   suffixes=(None, '_gr'),
                                   validate='1:1')

In [23]:
for stat_group in [traditional, advanced, four_factors, misc, scoring,
                   opponent, speed_distance, shooting, opponent_shooting, hustle]:
    side_df = side_df.merge(stat_group,
                                   how='left',
                                   left_on=['team', 'pred_date'],
                                   right_on=['team', 'date'],
                                   suffixes=(None, '_nba'),
                                   validate='1:m')
    side_df = side_df.drop(columns=['date_nba', 'date_opp', 'team_opp'], errors='ignore')
    side_df = side_df.merge(stat_group,
                                   how='left',
                                   left_on=['opponent', 'pred_date'],
                                   right_on=['team', 'date'],
                                   suffixes=(None, '_opp'),
                                   validate='1:m')
    side_df = side_df.drop(columns=['date_nba', 'date_opp', 'team_opp'], errors='ignore')

### Select Dates - Side Df

In [24]:
side_df = side_df[side_df['date'].between('2022-05-12', '2022-05-21', inclusive=False)]

# Concat Dataframes

In [25]:
side_df = side_df.drop(columns=['time', 'home_team_full_name',
                                'away_team_full_name', 'open_line_away'])

In [26]:
side_df = side_df.rename(columns={'id_num': 'id',
                                    'home_score': 'score',
                                    'away_score': 'opponent_score',
                                    'link': 'game_url',
                                    'open_line_home': 'spread'})

In [27]:
side_df['result'] = side_df.apply(lambda x: 'W' if x['score'] > x['opponent_score'] else 'L', axis=1)

In [28]:
side_df['spread_result'] = side_df.apply(lambda x: 'W' if (x['score'] - x['opponent_score']) > -x['spread'] else 'L', axis=1)

In [29]:
full_dataset = pd.concat((side_df, historic_df), ignore_index=True)

# Create Unique Record ID

In [30]:
full_dataset['game_id'] = full_dataset['date'].apply(lambda x: x.strftime('%Y%m%d')) + full_dataset['team'] + full_dataset['opponent']

# Cleanup

## Remove Excess Columns

In [31]:
columns_to_drop = ['id', 'home_team_short_name', 'away_team_short_name', 'opponent_gr',
                   'home_team', 'away_team']

In [32]:
full_dataset = full_dataset.drop(columns=columns_to_drop)

## Rename Columns

In [33]:
column_rename_dict = {'date': 'game_date', 'team': 'home_team', 'opponent': 'away_team',
                      'score': 'home_score', 'opponent_score': 'away_score', 'result': 'home_result',
                      'game_url': 'covers_game_url', 'spread': 'home_spread',
                      'spread_result': 'home_spread_result',
                      "fanduel_line_home": "fd_line_home",
                      "fanduel_line_price_home": "fd_line_price_home",
                      "fanduel_line_away": "fd_line_away",
                      "fanduel_line_price_away": "fd_line_price_away",
                      "draftkings_line_home": "dk_line_home",
                      "draftkings_line_price_home": "dk_line_price_home",
                      "draftkings_line_away": "dk_line_away",
                      "draftkings_line_price_away": "dk_line_price_away",
                      "covers_home_consenses": "covers_consenses_home",
                      "covers_away_consenses": "covers_consenses_away"}

In [34]:
full_dataset = full_dataset.rename(columns=column_rename_dict)

## Reorder Columns

In [36]:
main_features = ['game_id',
                 'game_date',
                 'home_team',
                 'away_team',
                 'home_score',
                 'away_score',
                 'home_result',
                 'league_year',
                 'pred_date',
                 'home_spread',
                 'home_spread_result',
                 'covers_game_url',
                 "fd_line_home",
                 "fd_line_price_home",
                 "fd_line_away",
                 "fd_line_price_away",
                 "dk_line_home",
                 "dk_line_price_home",
                 "dk_line_away",
                 "dk_line_price_away",
                 "covers_consenses_home",
                 "covers_consenses_away"]

all_features = main_features + [i for i in list(full_dataset) if i not in main_features]

In [37]:
full_dataset = full_dataset[all_features]

# Select Dates

In [38]:
# Ignoring first 5 days due to missing data
date_mask_2022 = (
        full_dataset['pred_date'] >= pd.to_datetime("October 24, 2021")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 10, 2022"))

date_mask_2021 = (
        full_dataset['pred_date'] >= pd.to_datetime("December 27, 2020")) & (
            full_dataset['game_date'] <= pd.to_datetime("May 16, 2021"))

date_mask_2020 = (
        full_dataset['pred_date'] >= pd.to_datetime("October 27, 2019")) & (
            full_dataset['game_date'] <= pd.to_datetime("August 14, 2020"))

date_mask_2019 = (
        full_dataset['pred_date'] >= pd.to_datetime("October 21, 2018")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 10, 2019"))

date_mask_2018 = (
        full_dataset['pred_date'] >= pd.to_datetime("October 22, 2017")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 11, 2018"))

date_mask_2017 = (
        full_dataset['pred_date'] >= pd.to_datetime("October 29, 2016")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 12, 2017"))

date_mask_2016 = (
        full_dataset['pred_date'] >= pd.to_datetime("November 1, 2015")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 13, 2016"))

date_mask_2015 = (
        full_dataset['pred_date'] >= pd.to_datetime("November 2, 2014")) & (
            full_dataset['game_date'] <= pd.to_datetime("April 15, 2015"))

In [39]:
full_dataset = full_dataset[date_mask_2022 | date_mask_2021 | date_mask_2020 | date_mask_2019 | date_mask_2018 | date_mask_2017 | date_mask_2016 | date_mask_2015]

# Review Data

In [40]:
full_dataset.isna().sum()

game_id                           0
game_date                         0
home_team                         0
away_team                         0
home_score                        0
                               ... 
contested_3pt_vla_opp          2130
contested_3pt_vla_std_opp      2130
contested_shots_rank_opp       2130
contested_shots_vla_opp        2130
contested_shots_vla_std_opp    2130
Length: 1230, dtype: int64

In [41]:
full_dataset.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 11 to 20042
Data columns (total 1230 columns):
 #     Column                              Non-Null Count  Dtype         
---    ------                              --------------  -----         
 0     game_id                             9200 non-null   object        
 1     game_date                           9200 non-null   datetime64[ns]
 2     home_team                           9200 non-null   object        
 3     away_team                           9200 non-null   object        
 4     home_score                          9200 non-null   float64       
 5     away_score                          9200 non-null   float64       
 6     home_result                         9200 non-null   object        
 7     league_year                         9200 non-null   object        
 8     pred_date                           9200 non-null   datetime64[ns]
 9     home_spread                         9200 non-null   float64       
 

## Set Datatypes

In [42]:
for column in list(full_dataset):
    if full_dataset[column].dtype == 'float64':
        full_dataset[column] = pd.to_numeric(full_dataset[column], downcast='float')
    if full_dataset[column].dtype == 'int64':
        full_dataset[column] = pd.to_numeric(full_dataset[column], downcast='integer')

In [43]:
full_dataset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9200 entries, 11 to 20042
Data columns (total 1230 columns):
 #     Column                              Dtype         
---    ------                              -----         
 0     game_id                             object        
 1     game_date                           datetime64[ns]
 2     home_team                           object        
 3     away_team                           object        
 4     home_score                          float32       
 5     away_score                          float32       
 6     home_result                         object        
 7     league_year                         object        
 8     pred_date                           datetime64[ns]
 9     home_spread                         float32       
 10    home_spread_result                  object        
 11    covers_game_url                     object        
 12    fd_line_home                        float32       
 13    fd_line_pric

# Save to RDS

In [44]:
# full_dataset.to_sql(name='combined_nba_covers', con=connection, index=False, if_exists='replace')